In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from scipy.stats.mstats import winsorize
import pandas as pd
import numpy as np

df = pd.read_csv('./data/diabetes_012_health_indicators_BRFSS2015.csv')
print(f"Tamanho original do dataset: {len(df):,} amostras")

# Selecionar features
features = [
    'HighBP', 'HighChol', 'BMI', 'Smoker', 'Stroke',
    'HeartDiseaseorAttack', 'PhysActivity',
    'Veggies', 'HvyAlcoholConsump', 'GenHlth', 'MentHlth',
    'PhysHlth', 'DiffWalk', 'Age', 'Education', 'Income'
]

target = 'Diabetes_012'

# Colunas que precisam ser escaladas
cols_to_scale = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Education', 'Income']

# Copiar dataset para pré-processamento
df_processed = df[features + [target]].copy()

# Transformar a variável alvo em binária: 1 para pré-diabetes e diabetes, 0 para sem diabetes
df_processed['Diabetes_012'] = df['Diabetes_012'].replace({2:1})
df_processed = df_processed.rename(columns = {'Diabetes_012': 'Diabetes_binary'})

print(f"Distribuição original das classes:")
print(df_processed['Diabetes_binary'].value_counts(normalize=True))

# Aplicar padronização Z-score nas colunas selecionadas
scaler = StandardScaler()
df_processed[cols_to_scale] = scaler.fit_transform(df_processed[cols_to_scale])

# Separar conjuntos para classificação (agora com a variável alvo binária)
X_class = df_processed[features].values
y_class = df_processed['Diabetes_binary'].values

# Separar treino e teste (somente para classificação)
X_train, X_test, y_train, y_test = train_test_split(
    X_class, y_class, test_size=0.3, random_state=42, stratify=y_class
)

print(f"\n Divisão final dos dados:")
print(f"Treino: {X_train.shape[0]:,} amostras")
print(f"Teste: {X_test.shape[0]:,} amostras")
print(f"Features: {X_train.shape[1]} características")

Tamanho original do dataset: 253,680 amostras
Distribuição original das classes:
Diabetes_binary
0.0    0.842412
1.0    0.157588
Name: proportion, dtype: float64

 Divisão final dos dados:
Treino: 177,576 amostras
Teste: 76,104 amostras
Features: 16 características


In [11]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Aplicar SMOTE ao conjunto de treino
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Verificar nova distribuição
unique, counts = np.unique(y_resampled, return_counts=True)
for cls, count in zip(unique, counts):
    pct = count / sum(counts) * 100
    print(f'Classe {cls}: {count} exemplos ({pct:.2f}%)')

Classe 0.0: 149592 exemplos (50.00%)
Classe 1.0: 149592 exemplos (50.00%)


In [12]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import time
from sklearn.model_selection import GridSearchCV

# Grid Search para otimizar hiperparâmetros
print("Iniciando Grid Search para otimização de hiperparâmetros...")
start_time = time.time()

# Definir grid de parâmetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9, 12, 15],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Criar modelo base
xgb_base = XGBClassifier(
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

# Configurar Grid Search
grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

# Executar Grid Search
grid_search.fit(X_resampled, y_resampled)

# Obter melhor modelo
xgb_model = grid_search.best_estimator_

print(f"\nMelhores parâmetros encontrados: {grid_search.best_params_}")
print(f"Melhor score CV: {grid_search.best_score_:.4f}")

xgb_model.fit(X_resampled, y_resampled)

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo de treinamento: {training_time:.2f} segundos")

# Predições
y_pred_train = xgb_model.predict(X_resampled)
y_pred_test = xgb_model.predict(X_test)

# Probabilidades para ROC-AUC
y_proba_train = xgb_model.predict_proba(X_resampled)[:, 1]
y_proba_test = xgb_model.predict_proba(X_test)[:, 1]

# Métricas para conjunto de treino
print("\n=== MÉTRICAS DO CONJUNTO DE TREINO ===")
print(f"Accuracy: {accuracy_score(y_resampled, y_pred_train):.4f}")
print(f"Precision: {precision_score(y_resampled, y_pred_train):.4f}")
print(f"Recall: {recall_score(y_resampled, y_pred_train):.4f}")
print(f"F1-Score: {f1_score(y_resampled, y_pred_train):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_resampled, y_proba_train):.4f}")

print("\nRelatório de classificação (Treino):")
print(classification_report(y_resampled, y_pred_train))

print("Matriz de confusão (Treino):")
print(confusion_matrix(y_resampled, y_pred_train))

# Métricas para conjunto de teste
print("\n=== MÉTRICAS DO CONJUNTO DE TESTE ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_test):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_test):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba_test):.4f}")

print("\nRelatório de classificação (Teste):")
print(classification_report(y_test, y_pred_test))

print("Matriz de confusão (Teste):")
print(confusion_matrix(y_test, y_pred_test))

Iniciando Grid Search para otimização de hiperparâmetros...
Fitting 3 folds for each of 45 candidates, totalling 135 fits

Melhores parâmetros encontrados: {'learning_rate': 0.05, 'max_depth': 9, 'n_estimators': 100}
Melhor score CV: 0.8454
Tempo de treinamento: 83.68 segundos

=== MÉTRICAS DO CONJUNTO DE TREINO ===
Accuracy: 0.8897
Precision: 0.9052
Recall: 0.8705
F1-Score: 0.8875
ROC-AUC: 0.9592

Relatório de classificação (Treino):
              precision    recall  f1-score   support

         0.0       0.88      0.91      0.89    149592
         1.0       0.91      0.87      0.89    149592

    accuracy                           0.89    299184
   macro avg       0.89      0.89      0.89    299184
weighted avg       0.89      0.89      0.89    299184

Matriz de confusão (Treino):
[[135962  13630]
 [ 19371 130221]]

=== MÉTRICAS DO CONJUNTO DE TESTE ===
Accuracy: 0.8341
Precision: 0.4715
Recall: 0.4381
F1-Score: 0.4542
ROC-AUC: 0.8182

Relatório de classificação (Teste):
           